# MyGO：个人旅行足迹数据库

## 组队信息
- 孙少凡 2100013085

## 业务需求：
- 记录行程：用户可以创建行程，行程可以存在多位参与者、涉及多个地点，可以记录行程的起始日期和结束日期。
- 足迹发表：用户可以针对某个地点发表“足迹”（游记评价文字），足迹可以包括标题、正文、发布时间等信息。
- 信息交互：用户可以收藏和评论其它用户发表的足迹，也可以评论其它用户的评论。

## 实体：
- 用户表
- 行程表
- 地点表
- 足迹表
- 评论表
- 收藏表
- 用户行程表
- 行程地点表
其中前三项可以认为是强实体，后五项都需要依赖于前三项的某几项存在。

## 关系：
- 用户参与行程，该关系使用用户行程表存储，它会存储行程id和用户id。
- 行程包含地点，该关系使用行程地点表存储，它会存储行程id和地点id。
- 用户位于地点发表足迹，足迹中会存储相应的用户id和地点id。
- 用户收藏足迹，该关系使用收藏表存储，它会存储用户id和足迹id。
- 用户对于足迹或评论发表评论，评论中会存储相应的用户id、足迹id和父评论id。

## ER图：
![](./assets/er_fig.jpg)


## 用户功能

### 一、数据表结构

```sql 
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT NOT NULL UNIQUE,
    email TEXT NOT NULL UNIQUE
)
```

### 二、界面总览

![user_management.png](assets/user_management.png)

展示所有用户、添加用户、删除用户。

### 三、核心功能接口

- 查询所有用户：`get_all_users(self)`
- 创建新用户：`create_user(self, username, email)`
- 删除用户：`delete_user(self, user_id)`

## 行程功能

### 一、数据表结构

```sql 
CREATE TABLE trips (
    trip_id INTEGER PRIMARY KEY AUTOINCREMENT,
    start_day DATE NOT NULL,
    end_day DATE NOT NULL CHECK(end_day > start_day)
)

CREATE TABLE trip_participants (
    user_id INTEGER NOT NULL REFERENCES users(user_id) ON DELETE CASCADE,
    trip_id INTEGER NOT NULL REFERENCES trips(trip_id) ON DELETE CASCADE, 
    PRIMARY KEY (user_id, trip_id)
)

CREATE TABLE trip_locations (
    location_id INTEGER NOT NULL REFERENCES locations(location_id) ON DELETE CASCADE,
    trip_id INTEGER NOT NULL REFERENCES trips(trip_id) ON DELETE CASCADE, 
    PRIMARY KEY (location_id, trip_id)
)             
```

### 二、界面总览

![trip_list.png](assets/trip_list.png)

展示所有行程。

![trip_create.png](assets/trip_create.png)

根据提供信息创建行程。

![trip_search.png](assets/trip_search.png)

行程筛选界面。

![trip_search_result.png](assets/trip_search_result.png)

在给出了一定信息后行程筛选返回的结果。

### 三、核心功能接口

- 查询所有行程：`get_all_trips(self)`
- 创建新行程：`create_trip(self, participants, start_day, end_day, location_ids)`
- 删除行程：`delete_trip(self, trip_id)`
- 根据条件筛选行程：`get_trips_by_filters(self, participants, start_after=None, start_before=Nont, end_after=None, arrived_locations=None)`

## 足迹功能

### 一、数据表结构
```sql
CREATE TABLE locations (
    location_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    address TEXT,
    type TEXT CHECK(type IN ('attraction','restaurant','transport'))
)

CREATE TABLE footprints (
    footprint_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    content TEXT,
    image_url TEXT,
    created_at DATETIME NOT NULL,
    user_id INTEGER NOT NULL REFERENCES users(user_id),
    location_id INTEGER NOT NULL REFERENCES locations(location_id)
)
```

### 二、核心接口说明

#### 1. 创建足迹
```python
def create_footprint(self, user_id, title, content, location_id):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        try:
            cursor.execute('''
                INSERT INTO footprints 
                (title, content, image_url, created_at, user_id, location_id)
                VALUES (?, ?, ?, datetime('now'), ?, ?)
            ''', (title, content, f'img_{int(time.time())}.jpg', user_id, location_id))
            conn.commit()
            return cursor.lastrowid
        except sqlite3.Error as e:
            print(f"创建足迹失败: {str(e)}")
            return None
```

#### 参数说明：
| 参数名       | 类型   | 必填 | 说明                |
|--------------|--------|------|---------------------|
| user_id      | int    | 是   | 用户ID（外键约束） |
| title        | str    | 是   | 足迹标题            |
| content      | str    | 否   | 图文内容            |
| location_id  | int    | 是   | 地点ID（外键约束） |

---

#### 2. 多条件查询
```python
def get_footprints_by_filters(self, username=None, location_name=None, 
                            location_types=None, created_after=None, 
                            created_before=None):
```

#### 查询条件示例：
```python
# 查询用户名为"test"、地点类型为景点、创建时间在2024年后的足迹
db.get_footprints_by_filters(
    username="test",
    location_types=["attraction"],
    created_after="2024-01-01 00:00:00"
)
```

#### 返回数据结构：
```json
[
    {
        "footprint_id": 1,
        "title": "长城游记",
        "content": "雄伟的古代防御工程",
        "image_url": "img_1620000000.jpg",
        "created_at": "2024-01-20 14:30",
        "user_id": 1,
        "location_id": 1,
        "location_name": "Great Wall",
        "location_type": "attraction",
        "username": "test_user"
    }
]
```

#### 3. 足迹详情查询
```python
def get_footprint_detail(self, footprint_id):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('''
            SELECT f.*, l.name as location_name, u.username 
            FROM footprints f
            JOIN locations l ON f.location_id = l.location_id
            JOIN users u ON f.user_id = u.user_id
            WHERE f.footprint_id = ?
        ''', (footprint_id,))
        row = cursor.fetchone()
        if row:
            return {
                'footprint_id': row[0],
                'title': row[1],
                'content': row[2],
                'image_url': row[3],
                'created_at': row[4],
                'user_id': row[5],
                'location_id': row[6],
                'location_name': row[7],
                'username': row[8]
            }
        return None
```

#### 功能特性
| 特性	| 说明 |
|-------|-------|
|精确查询 | 通过footprint_id精准定位单条足迹记录|
|关联数据获取 | 自动获取关联的用户名和地点名称 |
|空值处理 | 当ID不存在时返回None |

#### 请求示例
```python
# 查询ID为123的足迹详情
detail = db.get_footprint_detail(123)
print(detail['location_name'])  # 输出"Great Wall"
```

#### 4. 足迹信息更新
```python
def update_footprint(self, footprint_id, title=None, content=None, location_id=None):
    with self._get_connection() as conn:
        cursor = conn.cursor()
        try:
            update_fields = []
            params = []
            
            if title is not None:
                update_fields.append("title = ?")
                params.append(title)
            if content is not None:
                update_fields.append("content = ?")
                params.append(content)
            if location_id is not None:
                update_fields.append("location_id = ?")
                params.append(location_id)
                
            if not update_fields:
                raise ValueError("至少需要提供一个更新字段")
                
            params.append(footprint_id)
            query = f'''
                UPDATE footprints SET
                {', '.join(update_fields)}
                WHERE footprint_id = ?
            '''
            cursor.execute(query, params)
            conn.commit()
            return cursor.rowcount > 0
        except sqlite3.Error as e:
            conn.rollback()
            print(f"更新失败: {str(e)}")
            return False
```

#### 更新规则
| 参数 | 类型 | 约束 |
|------|------|------|
|footprint_id|int|必填，目标足迹ID|
|title|str|可选，新标题（非空）|
|content|str|可选，新内容（允许空串）|
|location_id|str|可选，有效地点ID|

### 三、界面展示
![](./assets/footprint_management.png)

创建新足迹

![](./assets/footprint_create.png)

![](./assets/footprint_created.png)

筛选

![](./assets/footprint_search.png)

编辑
![](./assets/footprint_edit.png)
